In [1]:
import pandas as pd
df = pd.read_csv("draft_data_public_BRO_PremierDraft.csv", chunksize=45)

In [ ]:
# Get the last pick of last pack
deck_list = []

for chunk in df:
    (deck_list.append(chunk.iloc[[-1]]))

In [ ]:
# Concat and create new .csv file
df_deck = pd.concat(deck_list)
df_deck.to_csv("all_decks.csv")